In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tools
import math
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.base import BaseEstimator
from sklearn.preprocessing import FunctionTransformer,Imputer,OneHotEncoder
from sklearn.base import TransformerMixin, BaseEstimator
from collections import defaultdict
from sklearn_pandas import gen_features,DataFrameMapper, CategoricalImputer

%matplotlib inline

Using TensorFlow backend.


## Loading the dataset

In [3]:
train = pd.read_csv("https://datahack-prod.s3.amazonaws.com/train_file/train_u6lujuX_CVtuZ9i.csv")
test = pd.read_csv("https://datahack-prod.s3.amazonaws.com/test_file/test_Y3wMUE5_7gLdaTN.csv")

## Copying the dataset

In [18]:
train_original = train.copy()
test_original = test.copy()

---

## Quick Data Exploration

### View first 10 rows

In [5]:
train.head(10)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LP001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


### Structure

#### Dataset

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               601 non-null object
Married              611 non-null object
Dependents           599 non-null object
Education            614 non-null object
Self_Employed        582 non-null object
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           592 non-null float64
Loan_Amount_Term     600 non-null float64
Credit_History       564 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 62.4+ KB


It can be infer from the above output that all the variables of type non-null object except Loan_ID should be converted to category type. The shape of the data set is:

In [7]:
train.shape

(614, 13)

#### Test

In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
Loan_ID              367 non-null object
Gender               356 non-null object
Married              367 non-null object
Dependents           357 non-null object
Education            367 non-null object
Self_Employed        344 non-null object
ApplicantIncome      367 non-null int64
CoapplicantIncome    367 non-null int64
LoanAmount           362 non-null float64
Loan_Amount_Term     361 non-null float64
Credit_History       338 non-null float64
Property_Area        367 non-null object
dtypes: float64(3), int64(2), object(7)
memory usage: 34.5+ KB


`Test` doesn't contain loan_status variable and null values are present there in the dataset. The shape of the dataset is:

In [9]:
test.shape

(367, 12)

### Category types

In [10]:
cat_var_list = ['Gender','Married','Dependents','Education','Self_Employed',
                'Credit_History','Property_Area','Loan_Status']

Following are the variables of category types:
1. Gender
2. Married
3. Dependents
4. Education
5. Self_Employed
6. Property_Area
7. Loan_Status
8. Credit_History

### Summary

In [11]:
train.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


Following are the few inference made from the above Output:
1. Loan Amount has 22 missing values.
2. Loan Amount Term has 14 missing values.
3. Credit History has 50 missing values.
4. It can also be seen from the credit history that 84% of the people have credit_history.

### Non-Numeric Values

#### Counts

In [12]:
train[cat_var_list].count()

Gender            601
Married           611
Dependents        599
Education         614
Self_Employed     582
Credit_History    564
Property_Area     614
Loan_Status       614
dtype: int64

#### Unique Values

In [13]:
for col in cat_var_list:
    print(col+":",train[col].unique())

Gender: ['Male' 'Female' nan]
Married: ['No' 'Yes' nan]
Dependents: ['0' '1' '2' '3+' nan]
Education: ['Graduate' 'Not Graduate']
Self_Employed: ['No' 'Yes' nan]
Credit_History: [ 1.  0. nan]
Property_Area: ['Urban' 'Rural' 'Semiurban']
Loan_Status: ['Y' 'N']


Following are the few inferences made from the Output:
1. Gender has 13 missing values.
2. Married has 3 missing values.
3. Dependants has 15 missing vlaues.
4. Self_Employed has 31 missing values.
5. Credit_History has 50 missing values.

---